# Monte Carlo Sampling

Compare with:

+ Arc
+ Act 1 & 2
+ Demo (Pulsed and SS)

// also adds prototype reactor designs (scylla and charybdis)

## 0.0 Bootup Notebook

In [ ]:
# if Sys.CPU_CORES != nprocs()
#     rmprocs(collect(2:100))
#     addprocs(Sys.CPU_CORES-1)
# end

addprocs(10)

@everywhere using Fussy
Main.IJulia.set_current_module(Fussy)

return

In [ ]:
using Fussy
Main.IJulia.set_current_module(Fussy)

return

## 1.0 Set Defaults

In [ ]:
cur_decks = [
    :scylla, # pulsed
    :charybdis, # steady state
    :arc, :act_1, :act_2,
    :demo_steady, :demo_pulsed
]
  
# cur_params = [ 
#     :wave_theta, :H, :Q, :epsilon, :kappa_95, :delta_95,
#     :nu_n, :nu_T, :l_i, :N_G, :f_D, :Z_eff,
#     :max_beta_N, :max_q_95, :max_P_W, :eta_CD,
#     :B_CS, :tau_FT
# ]

cur_sensitivity = 0.35

cur_params = [ 
    :wave_theta, :H, :Q, :epsilon, :delta_95,
    :nu_n, :nu_T, :l_i, :N_G, :f_D, 
    :Z_eff, :eta_CD, :B_CS, :tau_FT
]

return

## 2.0 Define Make Function

In [ ]:
using FileIO
using JLD2

In [ ]:
function make_sampling(cur_deck)
    println(cur_deck)
    
    cur_file = "../data/samplings_$(cur_deck).jld2"
    
    if cur_deck == :demo_pulsed || cur_deck == :scylla || cur_deck == :demo_steady
        is_consistent = false
    else
        is_consistent = true
    end
    
    cur_sampling = nothing
    try
        cur_sampling = load(cur_file, "cur_sampling")
    catch
        cur_sampling = Sampling(cur_params; sensitivity=cur_sensitivity, deck=cur_deck, is_consistent=is_consistent, max_reactors=5000, run_count=0, batch_count=1)
    end
    
    println(length(cur_sampling.reactors))
    
    max_batch = 99

    for cur_batch in 1:max_batch
        ( length(cur_sampling.reactors) >= 1000 ) && break
        
        println("===========")
        println("  ", cur_batch, " / ", max_batch)
        println("-----------")

        tmp_sampling = nothing
        try
            tmp_sampling = Sampling(cur_params; sensitivity=cur_sensitivity, deck=cur_deck, is_consistent=is_consistent, max_reactors=200, run_count=4000, batch_count=3, max_time=120)
        catch cur_error
            println("**** top ****")
            println(cur_error)
            println("**** bot ****")
            continue
        end

        append!(cur_sampling.reactors, tmp_sampling.reactors)
        sleep(0.15)

        println("added: ", length(tmp_sampling.reactors), "  // now: ", length(cur_sampling.reactors))
        save(cur_file, Dict("cur_sampling" => cur_sampling))
    end

    println("\n done.")
    
    cur_sampling
end

return

## 3.0 Get Data

In [ ]:
isdefined(Fussy, :default_samplings) || ( default_samplings = Dict() )

for cur_deck in cur_decks
    haskey(default_samplings, cur_deck) && continue
    println(cur_deck)
    
    if cur_deck == :demo_pulsed || cur_deck == :scylla || cur_deck == :demo_steady
        is_consistent = false
    else
        is_consistent = true
    end
    default_samplings[cur_deck] = Sampling(cur_params; sensitivity=eps(), deck=cur_deck, is_consistent=is_consistent, max_reactors=1, run_count=1, batch_count=1, max_time=120)
    
    isempty(default_samplings[cur_deck].reactors) && println(":/")
end

return


In [ ]:
cur_samplings = Dict()

for cur_deck in cur_decks
    cur_samplings[cur_deck] = make_sampling(cur_deck)
end

isempty(cur_samplings) || 
    save("../data/samplings_again.jld2", Dict("cur_samplings" => cur_samplings))

return

## 4.0 Setup for Plots

In [ ]:
using Interact
using Plots
using StringCases

In [ ]:
cur_fields = [ 
    "T_bar", "n_bar", "I_P", "R_0", "B_0", "R_CS",
    "P_F", "f_IN", "f_BS", "f_CD", "W_M", "cost", "eta_CD",
    "norm_beta_N", "norm_q_95", "norm_P_E", "norm_P_W", "b", "c", "d", "P_E"
]

cur_fields = map(zz -> Symbol(zz), cur_fields)

append!(cur_fields, cur_params)

cur_fields = sort(cur_fields)

cur_x_list = deepcopy(cur_fields)
cur_y_list = deepcopy(cur_fields)

deleteat!(cur_x_list, find(cur_field -> cur_field == :B_0, cur_x_list))
deleteat!(cur_y_list, find(cur_field -> cur_field == :cost, cur_y_list))

deleteat!(cur_x_list, find(cur_field -> cur_field == :cost, cur_x_list))
deleteat!(cur_y_list, find(cur_field -> cur_field == :B_0, cur_y_list))

unshift!(cur_y_list, :B_0)
unshift!(cur_x_list, :cost)

unshift!(cur_x_list, :B_0)
unshift!(cur_y_list, :cost)


## 5.0 Make Plot GUI

In [ ]:
@manipulate for y in cur_y_list, x in cur_x_list, yscale=[:lin, :log], xscale=[:lin, :log], deck in cur_decks
    default_reactor = default_samplings[deck].reactors[1]
    default_x = getfield(default_reactor, x)
    default_y = getfield(default_reactor, y)
    default_c = default_reactor.cost
       
    work_reactors = deepcopy(cur_samplings[deck].reactors)
    sort!(work_reactors, by=(cur_work_reactor -> cur_work_reactor.cost))
    
    cur_half_count = count(cur_work_reactor -> cur_work_reactor.cost <= default_c, work_reactors)
    cur_half_count = max( cur_half_count , Int(ceil(length(work_reactors)/6)) )
    work_reactors = work_reactors[1:min(length(work_reactors),2*cur_half_count)]
    
    cur_diff = abs(default_c - work_reactors[1].cost) * 1.1
    filter!(cur_work_reactor -> default_c - cur_diff <= cur_work_reactor.cost <= default_c + cur_diff, work_reactors)
    
    cur_x = map(cur_reac -> getfield(cur_reac, x), work_reactors)
    cur_y = map(cur_reac -> getfield(cur_reac, y), work_reactors)
    
    cur_w = map(cur_reac -> isapprox(cur_reac.norm_P_W, 1.0, atol=1e-4) ? :circle : :diamond, work_reactors)
    
    cur_c = map(cur_reac -> -log(cur_reac.cost), work_reactors)
    
    cur_o = map(cur_reac -> log(cur_reac.cost), work_reactors)
    cur_o -= minimum(cur_o)
    
    cur_o ./= maximum(cur_o)
    
    cur_o *= -1
    cur_o += 1
    
    cur_o .*= 2
    cur_o += 2
    
    cur_o = map(round, cur_o)
    
    scatter(cur_x, cur_y, color=:viridis, markersize=cur_o, zcolor=cur_c, label="", markershape=cur_w)
    
    min_x = minimum(cur_x) < 0 ? minimum(cur_x)*1.3 : ( xscale == :lin ? 0 : minimum(cur_x)*0.7 )
    min_y = minimum(cur_y) < 0 ? minimum(cur_y)*1.3 : ( yscale == :lin ? 0 : minimum(cur_y)*0.7 )
    
    max_x = max(default_x, maximum(cur_x))*1.3
    max_y = max(default_y, maximum(cur_y))*1.3
    
    if xscale == :lin && startswith(string(x), "f_")
        min_x = 0
        max_x = 1.3
    end
    
    if yscale == :log && y == :cost
        min_y = 9e-4
        max_y = 3e-2
    end
    
    xlims!(min_x, max_x)
    ylims!(min_y, max_y)
    
    xlabel!(string(x))
    ylabel!(string(y))
    
    try
        if x == :T_bar
            med_x = 20
        else
            x_index = find(cur_param -> cur_param == x, cur_sampling.parameters)[1]

            med_x = cur_sampling.defaults[x_index]
        end
        
        bot_x = med_x * ( 1 - cur_sampling.sensitivity )
        top_x = med_x * ( 1 + cur_sampling.sensitivity )
        
        plot!([med_x, med_x, NaN, bot_x, bot_x, NaN, top_x, top_x], [min_y,max_y, NaN, min_y,max_y, NaN, min_y,max_y],color=:lightblue,label="")
    catch
    end
    
    try
        if y == :T_bar
            med_y = 20
        else
            y_index = find(cur_param -> cur_param == y, cur_sampling.parameters)[1]

            med_y = cur_sampling.defaults[y_index]    
        end
    
        bot_y = med_y * ( 1 - cur_sampling.sensitivity )
        top_y = med_y * ( 1 + cur_sampling.sensitivity )
        
        plot!([min_x,max_x, NaN, min_x,max_x, NaN, min_x,max_x], [med_y, med_y, NaN, bot_y, bot_y, NaN, top_y, top_y],color=:lightblue,label="")
    catch
    end
    
    ( xscale == :log ) && plot!(xscale=:log10)
    ( yscale == :log ) && plot!(yscale=:log10)
    
    plot!(color=:default)
    plot!([min_x, max_x, NaN, default_x, default_x], [default_y, default_y, NaN, min_y, max_y], color=2, label="")
    
    if any(tmp_w -> tmp_w == :circle, cur_w)
        scatter!([],[],markershape=:circle, label="Beta-Wall", color=:grey)
    end
    if any(tmp_w -> tmp_w == :diamond, cur_w)
        scatter!([],[],markershape=:diamond, label="Beta-Kink", color=:grey)
    end
    
    title!(join(map(capitalize, split(string(deck), "_")), " "))
    plot!()
end